# Parse Document

In [1]:
import data_formats
import entities
import pickle
import pydantic as pyd
import pypdf
import os
import re
import time
import typing

DatasetEnvVarName = 'DATASET_ROOT'
assert DatasetEnvVarName in os.environ
DatasetEnvVar= os.environ[DatasetEnvVarName]

pdfFilePath = os.path.join(DatasetEnvVar,'Polizia_mortuaria.pdf')
assert os.path.isfile(pdfFilePath)

StorageEnvVarName = 'STORAGE_ROOT'
assert StorageEnvVarName in os.environ
StorageEnvVar= os.environ[StorageEnvVarName]

print(f'DatasetEnvVar:{DatasetEnvVar}')
print(f'StorageEnvVar:{StorageEnvVar}')

DatasetEnvVar:/home/minguzzi/repo/quiz_generator/dataset
StorageEnvVar:/home/minguzzi/repo/quiz_generator/storage


In [2]:
text= data_formats.readPdfAsText(pdfFilePath)
print(text)

Settore Servizi Demografici
REGOLAMENTO DI POLIZIA MORTUARIA
Approvato con deliberazione C.C. n. delCITTÀ di CASALE MONFERRATO –Regolamento Polizia Mortuaria
a
INDICE
CAPO I DISPOSIZIONI GENERALI Pag. 1
Art. 1 Dichiarazione di morte Pag. 1
Art. 2 Atto di morte “ 1
Art. 3 Causa di morte “ 1
Art. 4 Denuncia dei decessi “ 2
Art. 5 Visita necroscopica “ 2
Art. 6 Autorizzazione “ 2
Art. 7 Periodo di osservazione dei cadaveri “ 3
Art. 8 Manifestazioni di vita “ 3
Art. 9 Deposizioni di cadaveri nella cassa “ 3
Art. 10 Depositi in osservazione –Camera mortuaria “ 4
Art. 11 Attività funebre –Trasporto funebre “ 4
Art. 12 Percorsi dei trasporti funebri “ 5
Art. 13 Autopsie “ 5
CAPO II DISPOSIZIONI SPECIALI PER I CIMITERI Pag. 6
Art. 14 Vigilanza “ 6
Art. 15 Ubicazione e orari dei cimiteri “ 6
Art. 16 Permesso di seppellimento e consegna del cadavere nei cimiteri “ 6
Art. 17 Polizia dei cimiteri “ 6
Art. 18 Feretri per tumulazione “ 7
Art. 19 Salma non mineralizzata “ 7
Art. 20 Interventi manuten

In [3]:
pattern = r'(?=^Articolo\s*\d+$)'
articles = re.split(pattern, text, flags=re.MULTILINE)
# print('\nCHUNK:'.join(chunks[1:]))
len(articles)

62

In [4]:
for article in articles:
    print(article[0:80]+'\n')

Settore Servizi Demografici
REGOLAMENTO DI POLIZIA MORTUARIA
Approvato con delib

Articolo 1
DICHIARAZIONE DI MORTE
1. La dichiarazione di morte è fatta, non oltr

Articolo 2
ATTO DI MORTE
1. L' ufficiale dello Stato Civile redige l'atto di mor

Articolo 3
CAUSA DI MORTE
1. A norma dell'art. 103 T.U. Leggi Sanitarie, R.D. 27

Articolo 4
DENUNCIA DEI DECESSI
1. L’obbligo di denuncia del decesso è esteso an

Articolo 5
VISITA NECROSCOPICA
1. L'ufficiale dello Stato Civile accerta la mort

Articolo 6
AUTORIZZAZIONE
1. Sulla dichiarazione del medico, i ncaricato di cons

Articolo 7
PERIODO DI OSSERVAZIONE DEI CADAVERI
1. Nei casi in cui l'accertament

Articolo 8
MANIFESTAZIONI DI VITA
1. Durante il periodo di osservazione il corpo

Articolo 9
DEPOSIZIONI DI CADAVERI NELLA CASSA
1. Ogni cadavere, per essere tras

Articolo 10
DEPOSITI IN OSSERVAZIONE - CAMERA MORTUARIA
1. Nella camera mortuari

Articolo 11
ATTIVITA’ FUNEBRE- TRASPORTO FUNEBRE
1. Per attività fune bre si int

Articolo12
PERCO

Use langchain with a structured outoput to parse a text and divide into 



In [5]:
class Article(pyd.BaseModel):
    """ Un Articolo inizia con una riga che inizia con Articolo, seguita dal numero dell'articolo, esempio: Articolo 3, Articolo 5.
        la riga successiva a quella del numero dell'articolo contiene il titolo dell'articolo. 
        In seguito il testo dell'articolo, che comprende tutto il testo fino all'articolo successivo.
    """
    title: str= pyd.Field(description="Il titolo dell'articolo")
    text: str= pyd.Field(description="Il testo dell'articolo")
    references: typing.List[str] = pyd.Field(description="La lista di riferimenti nel testo, esempio: R.D. 27.7.1934 n. 1265, art. 254 T.U., artt. 81, 82 del D.p.r. n. 396/2000, art. 38 D.P.R. 10.9.1990 n. 285")

class Document(pyd.BaseModel):
    """ Un documento, e' diviso in capitoli marcati come CAPO.
    In ogni Capo si trova una successione di Articoli, seguiti dal testo dell'articolo.    
    """
    articles: typing.List[Article] = pyd.Field(description="La lista di Articoli contenuti nel documento")

    def addArticle(self, article: Article):
        self.articles.append(article)

In [6]:
from langchain_openai import ChatOpenAI
LlmModelName = 'gpt-4o'
llm = ChatOpenAI(model=LlmModelName, max_tokens=16384 )

In [7]:
structuredLlm = llm.with_structured_output( Article)

In [8]:
pattern = r'(?=^Articolo\s*\d+$)'
articles = re.split(pattern, text, flags=re.MULTILINE)
len(articles)

myDocument = entities.Document()

for indx,article in enumerate(articles[1:]):
    arguments= {'article':article}
    prompt= """
    Dato il seguente testo, individua l'articolo, il suo testo, ed i riferimenti contenuti in esso.
    Scarta l'indice iniziale.
    ---
    
    {article}
    """.format(**arguments)

    result = structuredLlm.invoke(prompt)

    myArticle = entities.Article(title=result.title, text=result.text, references= result.references)
    myDocument.addArticle( myArticle)   
    print(f'Article:{indx+1} terminated. Num articles: {myDocument.numArticles}')      
    time.sleep(1)
    documentFile= os.path.join( StorageEnvVar,'document.pickle' )
    with open(documentFile, 'wb') as f:
        pickle.dump( myDocument, f)    

Article:1 terminated. Num articles: 1
Article:2 terminated. Num articles: 2
Article:3 terminated. Num articles: 3
Article:4 terminated. Num articles: 4
Article:5 terminated. Num articles: 5
Article:6 terminated. Num articles: 6
Article:7 terminated. Num articles: 7
Article:8 terminated. Num articles: 8
Article:9 terminated. Num articles: 9
Article:10 terminated. Num articles: 10
Article:11 terminated. Num articles: 11
Article:12 terminated. Num articles: 12
Article:13 terminated. Num articles: 13
Article:14 terminated. Num articles: 14
Article:15 terminated. Num articles: 15
Article:16 terminated. Num articles: 16
Article:17 terminated. Num articles: 17
Article:18 terminated. Num articles: 18
Article:19 terminated. Num articles: 19
Article:20 terminated. Num articles: 20
Article:21 terminated. Num articles: 21
Article:22 terminated. Num articles: 22
Article:23 terminated. Num articles: 23
Article:24 terminated. Num articles: 24
Article:25 terminated. Num articles: 25
Article:26 termina

In [9]:
myDocument.numArticles

61

In [10]:
documentFile= os.path.join( StorageEnvVar,'document.pickle' )
with open(documentFile, 'wb') as f:
    pickle.dump( myDocument, f)

In [14]:
for indx,article in enumerate(myDocument.articles):
    print(f'{indx+1} {article.title}')

1 DICHIARAZIONE DI MORTE
2 ATTO DI MORTE
3 Causa di Morte
4 DENUNCIA DEI DECESSI
5 VISITA NECROSCOPICA
6 Autorizzazione
7 PERIODO DI OSSERVAZIONE DEI CADAVERI
8 MANIFESTAZIONI DI VITA
9 Deposition of Bodies in Coffins
10 DEPOSITI IN OSSERVAZIONE - CAMERA MORTUARIA
11 Attività Funebre - Trasporto Funebre
12 Percorsi dei Trasporti Funebri
13 AUTOPSIE
14 VIGILANZA
15 UBICAZIONE E ORARI DEI CIMITERI
16 Permesso di seppellimento e consegna del cadavere nei cimiteri
17 Polizia dei Cimiteri
18 FERETRI PER TUMULAZIONE
19 Salma Non Mineralizzata
20 Interventi Manutentivi e Festività di Ognissanti
21 Cura del Sepolcro e Oggetti Funerari
22 ISTANZE
23 DIVIETI
24 CAMPI COMUNI
25 FERETRI PER INUMAZIONI
26 Fosse per Inumazioni
27 Lapidi ed arredi funerari per inumazione: Caratteristiche tecniche
28 Concessioni Aree Cimiteriali - Corrispettivo
29 Limitazioni nelle Concessioni di Aree per Tombe
30 Divieti
31 Durata delle Concessioni e Obblighi dei Concessionari
32 TERMINI DI COSTRUZIONE
33 Prescrizion

In [12]:
myDocument.articles[1].title

'ATTO DI MORTE'

In [13]:
myDocument.articles[0].text

"1. La dichiarazione di morte è fatta, non oltre le 24 ore dal decesso, all'ufficiale dello Stato Civile del luogo dove questa è avvenuta o, nel caso in cui tale luogo si ignor i, del luogo dove il cadavere è stato deposto.\n2. La dichiarazione è fatta da uno dei congiunti o da una persona convivente con il defunto o da un loro delegato o, in mancanza, da persona informata del decesso.\n3. In caso di morte in un ospedale, casa di cura o di riposo, collegio, istituto o qualsiasi altro stabilimento, il direttore, o chi ne è stato delegato dall'amministrazione, deve trasmettere avviso della morte, nel termine fissato dal comma 1, all'ufficiale dello Stato Civile, con le indicazioni stabilite nell’art. 73 del D.p.r. n. 396/2000."